In [1]:
%load_ext autoreload
%autoreload 2

In [39]:
sys.path.append('/reg/d/psdm/tmo/tmox51020/results/paris/')
sys.path.append('/reg/d/psdm/tmo/tmox51020/results/paris/recon')
import torch
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy.io as scio
from torch.autograd import Variable
from torch.autograd import grad
from torch import Tensor
import scipy.ndimage as scimg
from scipy.interpolate import interp1d
import sys
sys.path.append('/reg/neh/home/tdd14/modules/cart2pol/cart2pol')
import cart2pol
from cart2pol import PolarRebin
from torchrecon_funcs_cal import *
import os
from scipy.ndimage import gaussian_filter as gf
from scipy.ndimage import median_filter as mf

In [11]:
from matplotlib import cm
from matplotlib.colors import ListedColormap
my_cmap = cm.get_cmap('jet')
xr = np.linspace(0, 1, 2000)
my_cmap = my_cmap(xr)
my_cmap[0:2,-1] = 0
my_cmap = ListedColormap(my_cmap)

def FWHM(t, spec): 
    peakMax = np.max(spec)
    indMax = np.where(spec == peakMax)[0] #should be center since rolled
    if len(indMax)>1: indMax = np.max(indMax)
    t1 = 0
    t2 = 0
    
    for i in range(int(len(t)/2)):
        if t2 != 0 and t1 != 0:
            break
        if i < indMax: 
            if spec[i] >= 0.5*peakMax and t1 == 0:
                t1 = t[i]
            if spec[-i] >= 0.5*peakMax and t2 == 0:
                t2 = t[-i]  
    return t2-t1

def gaus(x, a, mu, sig):
    return a*1/sig/np.sqrt(2*np.pi)*np.exp(-0.5*((x-mu)/sig)**2)


def gaus_fit(t,spec):
    #spec = Et*Et.conj()
    popt, pcov = curve_fit(gaus,t,spec, bounds=([0.1, -50, 0.01], [100, 50, 30]))
    a, mu, sig = popt
    return a, mu, sig #FWHM is 2.355*sig
def FWHM2(t, spec): 
    spec = spec/np.sum(spec)
    peakMax = np.max(spec)
    indMax = np.where(spec == peakMax)[0] 
    if len(indMax)>1: indMax = np.max(indMax)
    else: indMax = indMax[0]
    t1 = 0
    t2 = 0
    ind1 = 0
    ind2 = 0
    
    for i in range(int(len(t))):
        if t2 != 0 and t1 != 0:
            break
        if i < indMax:
            #print(i, np.trapz(spec[-i:]))
            
            if (np.trapz(spec[:i]) >= 0.25) & (t1 == 0):
                t1 = t[i]
                ind1 = i
            elif (np.trapz(spec[-(i+1):]) >= 0.25) & (t2 == 0):
                t2 = t[-(i+1)]  
                ind2 = -(i+1)
    return t2-t1 #, ind1, ind2

In [22]:
def make_MM(x0):
    xshift, yshift =  x0[8], x0[9] 

    # Coordinates that you want to Perspective Transform
    pts1 = np.float32([[0,0],\
                       [0,1024],\
                       [1024,0],\
                       [1024, 1024]])


    # Size of the Transformed Image
    pts2 = np.float32([[0+xshift-x0[0],0 + yshift-x0[1]],\
                       [0+xshift-x0[2],1024 + yshift-x0[3]],\
                       [1024+xshift-x0[4],0 + yshift-x0[5]],\
                       [1024+xshift-x0[6],1024 + yshift-x0[7]]])

    MM = cv2.getPerspectiveTransform(pts1,pts2)
    return MM


maskscale = np.zeros((1024,1024))
X,Y=np.meshgrid(np.arange(1024),np.arange(1024))
rs=np.sqrt((X-512)**2+(Y-512)**2)
maskscale [(rs>120) & (rs<270)]=1

def make_quads(x0):
    quads=np.ones((1024,1024))
    quads[:512,:512]= x0[0]
    quads[:512,512:]= x0[1]
    quads[512:,:512]= x0[2]
    quads[512:,512:]= x0[3]
    return quads

In [8]:
# #######
Ups = [0]
mfn = 25 
gfn = 15 
gfsig = 0.985 
runnum = 112
gfcart = 0.25 #applied to the simulated image to try to make it match the measured
mfcart = 1
thresh = 0.0006

In [33]:
eshift = 0 #0 #2 #1 #0 # 3.9/4.8*3 #6/4.02 #0 #4/4.9*3 #0 #opposite sign of dw
sc = 1 #3.9/4.8 #4.02/4.99 #4/4.9 #3.1/4.8
#k = 0
scale_bg = 1

In [19]:
n = 0
mdat = np.load('/cds/home/p/paris/reconVMI_clean/recon X510 Dec/Ms_Jul30_112_0.npy', allow_pickle = True)[()]
specdat = np.load('/cds/home/p/paris/reconVMI_clean/recon X510 Dec/specs_Jul30_112_0.npy', allow_pickle = True)
bgdat = np.load('/cds/home/p/paris/reconVMI_clean/recon X510 Jan/bgdat_run_112_Jul28.npy')
bins = bgdat['bins']          

In [29]:
basesdir = '/reg/d/psdm/tmo/tmox51020/results/paris/circularpol_vNbases_dipole_Feb/'
PR = PolarRebin('/reg/d/psdm/tmo/tmox51020/results/paris/recon/PR_c32_r32_th32.h5') 


In [30]:
Bps = []
alphas = []
vNaxiss = []

Np = 128
N_w=6
N_t=6 
#energy_x = 54 #eV
hbar=6.6e-16 #eV*s
Nb=N_t*N_w


Up = Ups[0] 
if Up == 0: Up = int(0)
filename = 'Bpbasis_Np' + str(Np) + '_Nw' + str(N_w) +'_Nt' + str(N_t) + '_Up' + str(Up)+'.npy'
br = np.load(basesdir+filename, allow_pickle=True)
Breal = br[()]['Breal']
Bimag = br[()]['Bimag']
breal = br[()]['breal']
bimag = br[()]['bimag']
vNaxis = br[()]['vNaxis']
alpha = br[()]['alpha']
Bp_basis = Breal+1j*Bimag

#rebin to 64x64
tempr = np.zeros((Breal.shape[0], 64**2))
tempi = np.zeros((Breal.shape[0], 64**2))
for i in range(len(Breal)):
    tempr[i,:] = np.reshape(rebin(np.reshape(Breal[i,:], [128,128])), [64**2])
    tempi[i,:] = np.reshape(rebin(np.reshape(Bimag[i,:], [128,128])), [64**2])

Breal = tempr
Bimag = tempi

alphw = np.zeros(alpha['t_sample'].shape, dtype = complex)
for i in range(len(alphw)):
    f = interp1d(vNaxis['t_sample'], alpha['t_sample'][i], fill_value = 0, bounds_error = False)
    N = 2801 #1024
    ts = np.linspace(vNaxis['t_sample'][0],-vNaxis['t_sample'][0], N)
    alphw[i] = np.fft.ifftshift(np.fft.ifft(f(ts)))


# alphw = np.zeros(alpha['t_sample'].shape, dtype = complex)
# for i in range(len(alphw)):
#     alphw[i] = np.fft.ifftshift(np.fft.ifft(alpha['t_sample'][i]))

alphaw_r=Variable(Tensor(np.real(alphw)))#real part
alphaw_i=Variable(Tensor(np.imag(alphw)))#imaginary part

Np = 64
for j in range(Breal.shape[0]):
    temp=np.reshape(Breal[j,:],[Np,Np])
    polimg = PR.cart2pol(temp,32,32)
    bb = np.zeros((64,64))
    bb[1:64,1:64]= PR.pol2cart(scimg.gaussian_filter(polimg,[gfsig,0]))
    Breal[j,:]=Variable(Tensor(np.reshape(bb*mask,newshape=(1,Np**2))))
    #temp=scimg.filters.gaussian_filter(temp,0.8889)
    #Bpr[j,:]=Variable(Tensor(np.reshape(temp,newshape=(1,Np**2))))
    temp=np.reshape(Bimag[j,:],[Np,Np])
    polimg = PR.cart2pol(temp,32,32)
    bb = np.zeros((64,64))
    bb[1:64,1:64]= PR.pol2cart(scimg.gaussian_filter(polimg,[gfsig,0]))
    Bimag[j,:]=Variable(Tensor(np.reshape(bb*mask,newshape=(1,Np**2))))
    #temp=scimg.filters.gaussian_filter(temp,0.8889)
    #Bpi[j,:]=Variable(Tensor(np.reshape(temp,newshape=(1,Np**2))))
    
Bpr=Variable(Tensor(Breal))
Bpi=Variable(Tensor(Bimag))

In [ ]:
torch.manual_seed(1)

m = mdat[n]

x,y, = 530,512
X,Y=np.meshgrid(np.arange(1024),np.arange(1024))
rs=np.sqrt((X-x)**2+(Y-y)**2)
mask=np.zeros((1024,1024))
mask[rs>1*48]=1

qqs = bgdat['quads'][np.argmax(bins-np.sum(m*mask)>0)-1]
if np.sum(qqs) == 0: print('quads no good')
resid = bgdat['bg_im'][np.argmax(bins-np.sum(m*mask)>0)-1]
quads = make_quads(qqs)
ims2 = m*quads
ims2 = mf(ims2, mfn)
ims2 = gf(ims2, gfn)
ims2= rebin(rebin(ims2,4),4)
resid_scale = resid/np.mean(resid*rebin(rebin(maskscale,4),4))*np.mean(ims2*rebin(rebin(maskscale,4),4))

ims2 = ims2 - resid_scale*scale_bg
ims2 = gf(ims2,.25)


ims2 = ims2*rebin(rebin(maskhm,4),4)*250/np.max(ims2)
ims2[ims2<thresh] = 0


X,Y=np.meshgrid(np.arange(1024),np.arange(1024))
rs=np.sqrt((X-512)**2+(Y-512)**2)
maskhm=np.zeros((1024,1024))
maskhm[(rs>325-30) & (rs<420)]=1
mask = rebin(rebin(maskhm,4),4)
m = ims2*mask

spec = specdat[n]
dat = {}
dat['thresh'] = thresh
dat['gfsig'] = gfsig
dat['gfcart'] = gfcart
dat['mfn'] = mfn
dat['gfn'] = gfn
dat['eshift'] = eshift
dat['sc'] = sc

M=Variable(Tensor(m))
M=M/torch.sum(M)
#M=M.transpose(0,1)
t=Variable(Tensor(vNaxis['t']))
ts=Variable(Tensor(vNaxis['t_sample']))
#ts=torch.linspace(t[0][0],t[0][-1],int(1e3))
w=Variable(Tensor(vNaxis['w']))
alphat_r=Variable(Tensor(np.real(alpha['t_sample'])))#real part
alphat_i=Variable(Tensor(np.imag(alpha['t_sample'])))#imaginary part

N = 2801
xf = np.arange(-N/2-1/2,N/2-1/2,1)/np.abs(vNaxis['t_sample'][0])/2
#xf = np.arange(-N/2+1/2,N/2+1/2,1)/((vNaxis['t_sample'][-1] - vNaxis['t_sample'][0])) #using tsample range
eV = 1239.84*(2*np.pi*xf)/2.9979E8/(2*np.pi)*1e-9 
#Interpolate spec outside of the loop to match eV
xeV = pix_2_eV(np.arange(1024), pixpereV = 22, spectra_hw0 = 0, spectra_pix0 = 512)
xeV = xeV - eshift #2.5 #3 #know this from previous calibration
f = interp1d(xeV,spec, bounds_error=False, fill_value = 0)
#spec_in = f(eV[1340:-1340])
spec_in = f(eV[1200:-1200])

##### new cal
xxx1 = np.arange(len(spec_in))
f = interp1d(xxx1, spec_in, fill_value = 0, bounds_error = False)
xs = np.linspace(sc*(xxx1[0] -np.argmax(spec_in)), sc*(xxx1[-1]-np.argmax(spec_in)), len(xxx1))+np.argmax(spec_in)
spec_in = f(xs)

####


spec_in= Variable(Tensor(spec_in))
spec_in=spec_in/torch.max(spec_in)

#torch.manual_seed(42)
#Q=torch.rand(1,2*Nb-1,requires_grad=True)
q = np.zeros((1,2*Nb-1))
q[:,1] = 1 #just doing 1 seed right now
Q = Tensor(q)
Q.requires_grad = True

n_epochs=1.3e3
lr=1e-2
optimizer=torch.optim.Adam([Q],lr=lr)
costs = []
costM = []
costW = []
costS = []
costQ = []
eps = []
M_wght = 1
Spec_wght = 15 #15 #12 #1 #0.75 #0.8
Smooth_wght = 0.00001
for epoch in range(int(n_epochs)):
    ll=loss_tot_noQ(M,Q,Nb,Bpr,Bpi,spec_in, alphaw_r, alphaw_i,M_wght,Spec_wght,Smooth_wght,thresh)
    loss_tot_noQ(M,Q,Nb,Bpr,Bpi,spec_in, alphaw_r, alphaw_i,M_wght,Spec_wght,Smooth_wght,thresh).backward()

    optimizer.step()
    optimizer.zero_grad()
    if epoch%100==0:
        print(epoch)
        eps.append(epoch)
    if ll < lr:
        break


dat['Qs'] = Q[0].detach().numpy()

In [ ]:
print(thresh,eshift,sc)

In [ ]:
#plot for ppt

fig,axs = plt.subplots(nrows = 1, ncols =6, figsize = (8,18))
m = mdat[n]
spec = specdat[n]   
       

ts= vNaxis['t_sample']
t1 = np.argmin((vNaxis['t_sample'] -vNaxis['t'][0])**2) 
t2 = np.argmin((vNaxis['t_sample'] -vNaxis['t'][-1])**2)


m = m/np.sum(m)
N = 2801
xf = np.arange(-N/2-1/2,N/2-1/2,1)/np.abs(vNaxis['t_sample'][0])/2
# xf = np.arange(-N/2+1/2,N/2+1/2,1)/((vNaxis['t_sample'][-1] - vNaxis['t_sample'][0])) #using tsample range
eV = 1239.84*(2*np.pi*xf)/2.9979E8/(2*np.pi)*1e-9 
#Interpolate spec outside of the loop to match eV
xeV = pix_2_eV(np.arange(1024), pixpereV = 22, spectra_hw0 = 0, spectra_pix0 = 512)
xeV = xeV # - eshift#3 #know this from previous calibration

Q = dat['Qs']            
N_basis = 36
eVshift_idx = 0
Q1=Q[0:N_basis]
Q1=np.reshape(Q1,[1,N_basis])
Q2=np.concatenate(([0],Q[N_basis:2*N_basis-1]))
Q2=np.reshape(Q2,[1,N_basis])
Qguess=Q1+1j*Q2
Qguess=np.reshape(Qguess,[N_basis,1])
Mgr=np.real(np.matmul(np.matmul(Qguess,Qguess.conj().transpose()).transpose().flatten(),Bp))
Mgr=Mgr/np.sum(Mgr)
Mgr[Mgr<thresh] = 0
Mgr=Mgr/np.sum(Mgr)

alpha_t = alpha['t_sample']/np.max(np.abs(alpha['t_sample'])) ##take out carrier
#alpha_t = alpha['t']/np.max(np.abs(alpha['t'])) #alpha_t = alpha_t*np.exp(-1.j*(energy_x)/hbar*np.tile(vNaxis['t'],(N_basis,1)))
Et = np.squeeze(np.matmul(Qguess.T,alpha_t))

f = interp1d(vNaxis['t_sample'],Et, fill_value = 0, bounds_error = False)
Et_l = f(np.linspace(vNaxis['t_sample'][0],-vNaxis['t_sample'][0], len(vNaxis['t_sample'])))
aghh = np.argmax(np.abs(Et_l))
Ew = np.fft.ifftshift(np.fft.ifft(np.roll(Et_l,-aghh))) #need this to get imag part, must be evenly spaced
#Ew = np.squeeze(np.matmul(Qguess.T,alphw))
EwEw = np.real(Ew*Ew.conj())
EwEw = EwEw[wr-eVshift_idx:-wr-eVshift_idx]
EwEw = EwEw/np.max(EwEw)
wphase = np.unwrap(np.angle(Ew))
wphase = wphase - np.mean(wphase)
Ew = np.squeeze(np.matmul(Qguess.T,alphw)) 
EwEw = np.real(Ew*Ew.conj())
EwEw = EwEw[wr-eVshift_idx:-wr-eVshift_idx]
EwEw = EwEw/np.max(EwEw)


#             aghh = np.argmax(np.abs(Et))
#             Ew = np.fft.ifftshift(np.fft.ifft(np.roll(Et,-aghh))) #need this to get imag part
#             #Ew = np.squeeze(np.matmul(Qguess.T,alphw))
#             EwEw = np.real(Ew*Ew.conj())
#             EwEw = EwEw[1340-eVshift_idx:-1340-eVshift_idx]
#             EwEw = EwEw/np.max(EwEw)
#             wphase = np.unwrap(np.angle(Ew))
#             wphase = wphase - np.mean(wphase)

phis = np.linspace(0,359,100)*np.pi/180

pscale = 1/np.max(m/np.sum(m*mask))
Mt = m/np.sum(m*mask)*pscale
Mp = pscale*np.reshape(Mgr/np.sum(mask*np.reshape(Mgr, [64,64])), [64,64])
p =axs[0].pcolormesh(Mt, cmap = my_cmap,vmax = np.max([np.max(Mp),1]))
axs[0].plot(32+ 20*np.cos(phis), 32+ 20*np.sin(phis), 'k-', alpha = 0.5)
axs[0].plot(32+ 18.5*np.cos(phis), 32+ 18.5*np.sin(phis), 'k--', alpha = 0.5)
plt.colorbar(p, ax = axs[0])

p =axs[1].pcolormesh(Mp, cmap = my_cmap,vmax = np.max([np.max(Mp),1]))#, vmin = np.min(m), vmax = np.max(m))
plt.colorbar(p, ax = axs[1])
phis = np.linspace(0,359,100)*np.pi/180
axs[1].plot(32+ 20*np.cos(phis), 32+ 20*np.sin(phis), 'k-', alpha = 0.5)
axs[1].plot(32+ 18.5*np.cos(phis), 32+ 18.5*np.sin(phis), 'k--', alpha = 0.5)

p = axs[2].pcolormesh((Mt-Mp)*mask, cmap = 'bwr')
axs[2].set_title('%1.4f' %np.sum(np.abs(Mt-Mp)*mask))
axs[2].plot(32+ 20*np.cos(phis), 32+ 20*np.sin(phis), 'k-', alpha = 0.5)
axs[2].plot(32+ 18.5*np.cos(phis), 32+ 18.5*np.sin(phis), 'k--', alpha = 0.5)
plt.colorbar(p, ax = axs[2])

f = interp1d(xeV,spec, bounds_error=False, fill_value = 0)
spec_in = f(eV[wr:-wr])

# ############# spec cal
# xxx1 = np.arange(len(spec_in))
# f = interp1d(xxx1, spec_in, fill_value = 0, bounds_error = False)
# xs = np.linspace((xxx1[0] -np.argmax(spec_in)), (xxx1[-1]-np.argmax(spec_in)), len(xxx1))+np.argmax(spec_in)
# spec_in = f(xs)
# #########

spec_in=spec_in/np.max(spec_in)
ax1 = axs[5].twinx()
ax1.plot(eV[wr:-wr], spec_in, color= 'tab:green', alpha = 0.5)
ax = axs[4].twinx()
ax.plot(eV[wr:-wr], spec_in, color= 'tab:green', alpha = 0.5)


axs[5].set_title('w')
axs[4].plot(np.linspace(eV[wr],eV[-wr], len(EwEw)), np.squeeze(EwEw))
ax.set_ylim(axs[4].get_ylim())
EtEt = np.abs(Et*Et.conj())
EtEt = EtEt/np.max(EtEt)
axs[3].plot(ts[t1:t2], EtEt[t1:t2],label = 'seed %1.0f' %seeds[seed_order[s]])
axs[3].grid()

            
axs[5].set_title('w phase')

axs[5].plot(eV[wr-eVshift_idx:-wr-eVshift_idx], wphase[wr-eVshift_idx:-wr-eVshift_idx])
mid = int(len(eV[wr-eVshift_idx:-wr-eVshift_idx])/4)

axs[3].set_title('t') 
axs[4].set_xlim(-12,12)
axs[5].set_xlim(-12,12)
        
plt.show()